# Модуль typing

https://habr.com/ru/company/lamoda/blog/432656/

Создан для того, чтобы отслеживать правильность типов передаваемых аргументов в функции. Аннотации функций появились в Python 3.6, модуль typing - в 3.7.

Аннотации типов призваны помочь программисту быстрее понять сигнатуру функции, но не предотвращают ошибок в передаваемых данных. Также они поддерживаются PyCharm'ом и линтерами.

In [1]:
def some_function(x, y, temp, value, processor):
    pass

In [2]:
def upper(s: str) -> str:
    print("entered")
    return s.upper()

upper(12342)

entered


AttributeError: 'int' object has no attribute 'upper'

Установим популярный статический анализатор mypy:

In [3]:
!pip install mypy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 4.0 MB/s eta 0:00:0000:0100:01
  Using cached mypy_extensions-0.4.3-py2.py3-none-any.whl (4.5 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


Код из предыдущей ячейки записан в файле 07/typing1.py. Попробуем прогнать его через mypy:

In [4]:
!mypy 07/typing1.py

07/typing1.py:5: error: Argument 1 to "upper" has incompatible type "int"; expected "str"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


### Алиасы типов

Благодаря возможностям библиотеки typing, можно часто используемые составные типы записывать в переменные:

In [5]:
from typing import List

Vector = List[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]


scale(2, [1, -4.2, 5.2])

[2, -8.4, 10.4]

## Структуры типов модуля typing

- `Any` - произвольный тип
- `List[int]` - список, который содержит только один тип данных int
- `Tuple[int, str]` - кортеж, который может содержать несколько типов данных
- `Union[str, bytes]` - можно подавать либо строку, либо bytes
- `Callable[[int, int], float]` - вызываемый объект, который на вход принимает два аргумента int, а возвращает float
- `Iterable[T]` - Iterable со значениями типа T
- `Mapping[K, V]`, `Dict[K, V]` - словарь с ключами типа K и значениями типа V
- `Awaitable[T_co]` - корутины
- `Type[T]` - тип/класс


- `Optional[T]` - показывает, что переменная может быть None

### Optional

По умолчанию аннотированный тип не может быть None:

In [ ]:
from typing import Optional

amount: int
amount = None  # Incompatible types in assignment (expression has type "None", variable has type "int")

price: Optional[int]
price = None

In [6]:
!mypy -c "amount: int = None"

<string>:1: error: Incompatible types in assignment (expression has type "None", variable has type "int")  [assignment]
Found 1 error in 1 file (checked 1 source file)


In [7]:
!mypy -c "from typing import Optional; amount: Optional[int] = None"

Success: no issues found in 1 source file


### Any

Если мы берем на себя ручную обработку типов, можем аннотировать переменную классом Any, тогда она сможет принимать любые значения

In [8]:
from typing import Any

amount: Any
amount = 1
amount = "Some"
amount = None

In [9]:
!mypy -c "from typing import Any; amount: Any; amount = 1; amount = 'Some'; amount = None"

Success: no issues found in 1 source file


### Union

Предназначен для случаев, когда можно использовать только некоторые типы

In [10]:
from typing import Union, Dict

def some_func(arg: Union[Dict[str, str], str]) -> int:
    return len(arg)


some_func({"a": "b"})
some_func("abc")
some_func({"a": 1})

1

In [11]:
!mypy 07/typing2.py

07/typing2.py:9: error: Dict entry 0 has incompatible type "str": "int"; expected "str": "str"  [dict-item]
Found 1 error in 1 file (checked 1 source file)


## Generic-типы

Иногда нужно просто указать, что данные должны быть однотипными, без жесткой фиксации типов. Для этого используется TypeVar:

In [ ]:
from typing import TypeVar, Generic

T = TypeVar("T")

class LinkedList(Generic[T]):
    data: T
    next: "LinkedList[T]"

    def __init__(self, data: T):
        self.data = data

head_int: LinkedList[int] = LinkedList(1)
head_int.next = LinkedList(2)
head_int.next = 2  # error: Incompatible types in assignment (expression has type "int", variable has type "LinkedList[int]")
head_int.data += 1
head_int.data.replace("0", "1")  # error: "int" has no attribute "replace"

head_str: LinkedList[str] = LinkedList("1")
head_str.data.replace("0", "1")

head_str = LinkedList[str](1)  # error: Argument 1 to "LinkedList" has incompatible type "int"; expected "str"

In [12]:
!mypy 07/typing3.py

07/typing3.py:14: error: Incompatible types in assignment (expression has type "int", variable has type "LinkedList[int]")  [assignment]
07/typing3.py:16: error: "int" has no attribute "replace"  [attr-defined]
07/typing3.py:21: error: Argument 1 to "LinkedList" has incompatible type "int"; expected "str"  [arg-type]
Found 3 errors in 1 file (checked 1 source file)


Еще один пример - пара объектов для любых конкретных типов:

In [ ]:
from typing import TypeVar, Generic

K = TypeVar('K')
V = TypeVar('V')

class Pair(Generic[K, V]):
    def __init__(self, key: K, value: V):
        self._key = key
        self._value = value

    @property
    def key(self) -> K:
        return self._key

    @property
    def value(self) -> V:
        return self._value


class IntPair(Pair[int, int]):
    pass

p = IntPair("1", "2")

## Cast

Иногда статический анализатор не может однозначно определить тип переменной. Чтобы показать анализатору, что возвращается действительно заявленный тип, можно в коде использовать функцию cast.

In [ ]:
from typing import List, cast

def find_first_str(a: List[object]) -> str:
    index = next(i for i, x in enumerate(a) if isinstance(x, str))
    return cast(str, a[index])

Это полезно для декораторов, поскольку анализатору может быть непонятно, что представляет собой обобщенный wrapper:

In [ ]:
MyCallable = TypeVar("MyCallable", bound=Callable)

def logged(func: MyCallable) -> MyCallable:
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)

    return cast(MyCallable, wrapper)

@logged
def mysum(a: int, b: int) -> int:
    return a + b

mysum(a=1)  # error: Missing positional argument "b" in call to "mysum"

# Decimal

Такая особенность встречается во многих языках программирования:

In [14]:
1.1 + 2.2

3.3000000000000003

In [15]:
0.1 + 0.1 + 0.1 - 0.3

5.551115123125783e-17

In [16]:
from decimal import Decimal

float(Decimal('1.1') + Decimal('2.2'))

3.3

In [17]:
float(Decimal('0.1') + Decimal('0.1') + Decimal('0.1') - Decimal('0.3'))

0.0

# Logging

https://habr.com/ru/post/144566/

Когда проект разрастается до определенной степени, появляется необзодимость в ведении журнала событий - лога. Это нужно, чтобы быстро понимать причины ошибок, улавливать нетипичное поведение программы, искать аномалии во входящих данных и т.д.

В Python есть встроенная библиотека, которая позволяет удобно логировать события. Изначально представлены 5 уровне логирования:

- debug - для отладки
- info - просто информационное сообщение
- warning - предупреждение
- error - ошибка
- critical - критическая ошибка

In [18]:
import logging

logging.debug("Сообщение для отладки")
logging.info("Самое обыкновенное информационное сообщение")
logging.warning("Предупреждение")
logging.error("Ошибка")
logging.critical("Полный крах")

ERROR:root:Ошибка
CRITICAL:root:Полный крах


Вывелись не все сообщения, поскольку по умолчанию уровень вывода сообщений - warning. Можем его поменять, но это нужно сделать до первого вызова вывода ошибки.

In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
logging.debug("Сообщение для отладки")
logging.info("Самое обыкновенное информационное сообщение")
logging.warning("Предупреждение")
logging.error("Ошибка")
logging.critical("Полный крах")

DEBUG:root:Сообщение для отладки
INFO:root:Самое обыкновенное информационное сообщение
ERROR:root:Ошибка
CRITICAL:root:Полный крах


Есть несколько встроенных в библиотеку значений, которые могут помочь сделать лог более подробным: 

<table class="docutils align-default">
<colgroup>
<col style="width: 18%">
<col style="width: 28%">
<col style="width: 53%">
</colgroup>
<thead>
<tr class="row-odd"><th class="head"><p>Attribute name</p></th>
<th class="head"><p>Format</p></th>
<th class="head"><p>Description</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>args</p></td>
<td><p>You shouldn’t need to
format this yourself.</p></td>
<td><p>The tuple of arguments merged into <code class="docutils literal notranslate"><span class="pre">msg</span></code> to
produce <code class="docutils literal notranslate"><span class="pre">message</span></code>, or a dict whose values
are used for the merge (when there is only one
argument, and it is a dictionary).</p></td>
</tr>
<tr class="row-odd"><td><p>asctime</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(asctime)s</span></code></p></td>
<td><p>Human-readable time when the
<a class="reference internal" href="#logging.LogRecord" title="logging.LogRecord"><code class="xref py py-class docutils literal notranslate"><span class="pre">LogRecord</span></code></a> was created.  By default
this is of the form ‘2003-07-08 16:49:45,896’
(the numbers after the comma are millisecond
portion of the time).</p></td>
</tr>
<tr class="row-even"><td><p>created</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(created)f</span></code></p></td>
<td><p>Time when the <a class="reference internal" href="#logging.LogRecord" title="logging.LogRecord"><code class="xref py py-class docutils literal notranslate"><span class="pre">LogRecord</span></code></a> was created
(as returned by <a class="reference internal" href="time.html#time.time" title="time.time"><code class="xref py py-func docutils literal notranslate"><span class="pre">time.time()</span></code></a>).</p></td>
</tr>
<tr class="row-odd"><td><p>exc_info</p></td>
<td><p>You shouldn’t need to
format this yourself.</p></td>
<td><p>Exception tuple (à la <code class="docutils literal notranslate"><span class="pre">sys.exc_info</span></code>) or,
if no exception has occurred, <code class="docutils literal notranslate"><span class="pre">None</span></code>.</p></td>
</tr>
<tr class="row-even"><td><p>filename</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(filename)s</span></code></p></td>
<td><p>Filename portion of <code class="docutils literal notranslate"><span class="pre">pathname</span></code>.</p></td>
</tr>
<tr class="row-odd"><td><p>funcName</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(funcName)s</span></code></p></td>
<td><p>Name of function containing the logging call.</p></td>
</tr>
<tr class="row-even"><td><p>levelname</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(levelname)s</span></code></p></td>
<td><p>Text logging level for the message
(<code class="docutils literal notranslate"><span class="pre">'DEBUG'</span></code>, <code class="docutils literal notranslate"><span class="pre">'INFO'</span></code>, <code class="docutils literal notranslate"><span class="pre">'WARNING'</span></code>,
<code class="docutils literal notranslate"><span class="pre">'ERROR'</span></code>, <code class="docutils literal notranslate"><span class="pre">'CRITICAL'</span></code>).</p></td>
</tr>
<tr class="row-odd"><td><p>levelno</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(levelno)s</span></code></p></td>
<td><p>Numeric logging level for the message
(<code class="xref py py-const docutils literal notranslate"><span class="pre">DEBUG</span></code>, <code class="xref py py-const docutils literal notranslate"><span class="pre">INFO</span></code>,
<code class="xref py py-const docutils literal notranslate"><span class="pre">WARNING</span></code>, <code class="xref py py-const docutils literal notranslate"><span class="pre">ERROR</span></code>,
<code class="xref py py-const docutils literal notranslate"><span class="pre">CRITICAL</span></code>).</p></td>
</tr>
<tr class="row-even"><td><p>lineno</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(lineno)d</span></code></p></td>
<td><p>Source line number where the logging call was
issued (if available).</p></td>
</tr>
<tr class="row-odd"><td><p>message</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(message)s</span></code></p></td>
<td><p>The logged message, computed as <code class="docutils literal notranslate"><span class="pre">msg</span> <span class="pre">%</span>
<span class="pre">args</span></code>. This is set when
<a class="reference internal" href="#logging.Formatter.format" title="logging.Formatter.format"><code class="xref py py-meth docutils literal notranslate"><span class="pre">Formatter.format()</span></code></a> is invoked.</p></td>
</tr>
<tr class="row-even"><td><p>module</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(module)s</span></code></p></td>
<td><p>Module (name portion of <code class="docutils literal notranslate"><span class="pre">filename</span></code>).</p></td>
</tr>
<tr class="row-odd"><td><p>msecs</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(msecs)d</span></code></p></td>
<td><p>Millisecond portion of the time when the
<a class="reference internal" href="#logging.LogRecord" title="logging.LogRecord"><code class="xref py py-class docutils literal notranslate"><span class="pre">LogRecord</span></code></a> was created.</p></td>
</tr>
<tr class="row-even"><td><p>msg</p></td>
<td><p>You shouldn’t need to
format this yourself.</p></td>
<td><p>The format string passed in the original
logging call. Merged with <code class="docutils literal notranslate"><span class="pre">args</span></code> to
produce <code class="docutils literal notranslate"><span class="pre">message</span></code>, or an arbitrary object
(see <a class="reference internal" href="../howto/logging.html#arbitrary-object-messages"><span class="std std-ref">Using arbitrary objects as messages</span></a>).</p></td>
</tr>
<tr class="row-odd"><td><p>name</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(name)s</span></code></p></td>
<td><p>Name of the logger used to log the call.</p></td>
</tr>
<tr class="row-even"><td><p>pathname</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(pathname)s</span></code></p></td>
<td><p>Full pathname of the source file where the
logging call was issued (if available).</p></td>
</tr>
<tr class="row-odd"><td><p>process</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(process)d</span></code></p></td>
<td><p>Process ID (if available).</p></td>
</tr>
<tr class="row-even"><td><p>processName</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(processName)s</span></code></p></td>
<td><p>Process name (if available).</p></td>
</tr>
<tr class="row-odd"><td><p>relativeCreated</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(relativeCreated)d</span></code></p></td>
<td><p>Time in milliseconds when the LogRecord was
created, relative to the time the logging
module was loaded.</p></td>
</tr>
<tr class="row-even"><td><p>stack_info</p></td>
<td><p>You shouldn’t need to
format this yourself.</p></td>
<td><p>Stack frame information (where available)
from the bottom of the stack in the current
thread, up to and including the stack frame
of the logging call which resulted in the
creation of this record.</p></td>
</tr>
<tr class="row-odd"><td><p>thread</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(thread)d</span></code></p></td>
<td><p>Thread ID (if available).</p></td>
</tr>
<tr class="row-even"><td><p>threadName</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">%(threadName)s</span></code></p></td>
<td><p>Thread name (if available).</p></td>
</tr>
</tbody>
</table>

Применяются они так:

In [1]:
import logging

logging.basicConfig(
    format='%(filename)s[LINE:%(lineno)d]# %(levelname)-8s [%(asctime)s]  %(message)s',
    level=logging.DEBUG
)

In [2]:
logging.debug("Сообщение для отладки")
logging.info("Самое обыкновенное информационное сообщение")
logging.warning("Предупреждение")
logging.error("Ошибка")
logging.critical("Полный крах")

<ipython-input-2-fe25b5922d11>[LINE:1]# DEBUG    [2022-12-02 00:48:13,623]  Сообщение для отладки
<ipython-input-2-fe25b5922d11>[LINE:2]# INFO     [2022-12-02 00:48:13,625]  Самое обыкновенное информационное сообщение
<ipython-input-2-fe25b5922d11>[LINE:3]# WARNING  [2022-12-02 00:48:13,625]  Предупреждение
<ipython-input-2-fe25b5922d11>[LINE:4]# ERROR    [2022-12-02 00:48:13,626]  Ошибка
<ipython-input-2-fe25b5922d11>[LINE:5]# CRITICAL [2022-12-02 00:48:13,627]  Полный крах


## Запись лога в файл

Конечно, просто выводить логи на экран - бессмысленная затея. Нужно сохранять их в файл:

In [1]:
import logging

logging.basicConfig(
    format='%(filename)s[LINE:%(lineno)d]# %(levelname)-8s [%(asctime)s]  %(message)s',
    level=logging.DEBUG,
    filename="log.txt",
    filemode="w"
)

logging.debug("Сообщение для отладки")
logging.info("Самое обыкновенное информационное сообщение")
logging.warning("Предупреждение")
logging.error("Ошибка")
logging.critical("Полный крах")

In [2]:
with open("log.txt") as f:
    print(f.read())

<ipython-input-1-62cc18cfba68>[LINE:10]# DEBUG    [2022-12-02 01:11:33,348]  Сообщение для отладки
<ipython-input-1-62cc18cfba68>[LINE:11]# INFO     [2022-12-02 01:11:33,349]  Самое обыкновенное информационное сообщение
<ipython-input-1-62cc18cfba68>[LINE:12]# WARNING  [2022-12-02 01:11:33,349]  Предупреждение
<ipython-input-1-62cc18cfba68>[LINE:13]# ERROR    [2022-12-02 01:11:33,349]  Ошибка
<ipython-input-1-62cc18cfba68>[LINE:14]# CRITICAL [2022-12-02 01:11:33,350]  Полный крах



## Несколько логгеров

Использование общей конфигурации для логов на весь проект - плохая идея, поскольку это влияет и на логи окружения, и всё сливается в одну кашу. Лучше завести для каждой отдельной части крупного приложения свой логгер.

In [3]:
import logging

# получим логгер для нашего приложения либо создадим новый, если он еще не создан (паттерн Синглтон)
logger = logging.getLogger("our_app_name")
logger.setLevel(logging.DEBUG)
# опишем, куда и как будем сохранять логи: зададим файл и формат
handler = logging.FileHandler('our_app_log.txt', 'a', 'utf-8')
formatter = logging.Formatter("%(filename)s[LINE:%(lineno)d]# %(levelname)-8s [%(asctime)s]  %(message)s")

# установим файлу нужный формат, а нужный файл - логгеру
handler.setFormatter(formatter)
logger.addHandler(handler)

# можно даже записывать сразу в несколько файлов
handler2 = logging.FileHandler('our_app_log2.txt', 'a', 'utf-8')
handler2.setFormatter(formatter)
logger.addHandler(handler2)

In [4]:
logger.info("Наш новый логгер работает")

In [6]:
with open("our_app_log2.txt", encoding="utf8") as f:
    print(f.read())

<ipython-input-4-c291bfcc7c8e>[LINE:1]# INFO     [2022-12-02 01:14:35,538]  Наш новый логгер работает



In [7]:
logger.handlers

[<FileHandler /Users/romanov/Documents/МГТУ/dc-web-developer-2022/01-python/week-8/our_app_log.txt (NOTSET)>,
 <FileHandler /Users/romanov/Documents/МГТУ/dc-web-developer-2022/01-python/week-8/our_app_log2.txt (NOTSET)>]

In [8]:
handler.close()
handler2.close()